In [191]:

# sourcery skip: for-index-underscore
import pandas as pd
import requests
from datetime import *
from bs4 import BeautifulSoup
import lxml.etree as et
import html
import re
from tqdm import tqdm
import collections

In [192]:
url = "https://www.tmz.com"
req = requests.get(url)
doc = BeautifulSoup(req.text)

In [193]:
raw_titles = doc.select("header > a > h2")

In [194]:
# Grabs all the headlines, appends period if doesn't end in !
count = 0
headlines = []
valid_punct = ["'", "!", "?"]
for title in tqdm(enumerate(raw_titles)):
    print("- - -")
    headline = raw_titles[count].get_text().strip().replace("\n", " ") + "."
    if headline[-2] in valid_punct:
        headline = headline[:-1]
    else: pass

    print(headline)
    headlines.append(headline)
    count += 1

16it [00:00, 66378.70it/s]

- - -
Jake Paul Hasim Rahman Jr. Fight Canceled ... Over Weight Issue.
- - -
Taylor Swift Fan Claim She Donated to Environment Charity??? Likely BS Amid Jet Backlash.
- - -
Dave Chappelle Tokes Up, Performs with Snoop ... Wifey Likes to Party Too!!!
- - -
President Biden I Got Covid Again ... After Days of Being Negative.
- - -
Kate Bush's 'Wuthering Heights' Fans Gather En Masse to Dance ... All These People in Red!!!
- - -
Arnold Schwarzenegger Shout-Out to My Late Mother ... Day Before My 75th Bday!!!
- - -
Stars and Scars You Be the judge.
- - -
Jon Stewart Calls BS on Ted Cruz ... You're Lying About the Vet Bill!!!
- - -
Ivana Trump Pics of Gravesite Go Viral ... Bare at DT's Golf Course.
- - -
Drake Brings Out Nelly Furtado at OVO Fest ... Crowd Loses It.
- - -
Britney Spears My Book is Done!!! ... But There's No Paper to Print it On!!!
- - -
IG Model Gena Tew AIDS Has Taken Away My Mobility, Left Me Blind in One Eye.
- - -
NY Gov. Hochul Orders Disaster Emergency ... As Monkeypo

In [195]:
headlines

['Jake Paul Hasim Rahman Jr. Fight Canceled ... Over Weight Issue.',
 'Taylor Swift Fan Claim She Donated to Environment Charity??? Likely BS Amid Jet Backlash.',
 'Dave Chappelle Tokes Up, Performs with Snoop ... Wifey Likes to Party Too!!!',
 'President Biden I Got Covid Again ... After Days of Being Negative.',
 "Kate Bush's 'Wuthering Heights' Fans Gather En Masse to Dance ... All These People in Red!!!",
 'Arnold Schwarzenegger Shout-Out to My Late Mother ... Day Before My 75th Bday!!!',
 'Stars and Scars You Be the judge.',
 "Jon Stewart Calls BS on Ted Cruz ... You're Lying About the Vet Bill!!!",
 "Ivana Trump Pics of Gravesite Go Viral ... Bare at DT's Golf Course.",
 'Drake Brings Out Nelly Furtado at OVO Fest ... Crowd Loses It.',
 "Britney Spears My Book is Done!!! ... But There's No Paper to Print it On!!!",
 'IG Model Gena Tew AIDS Has Taken Away My Mobility, Left Me Blind in One Eye.',
 'NY Gov. Hochul Orders Disaster Emergency ... As Monkeypox Cases Keep Stacking Up.',


In [197]:
# init dataframe of tags
tagframe = pd.DataFrame()
# select blob with tag
tags = doc.select("section.tag-cloud > ul > a")
# target tag str
tag = str(tag)[109:111]

tag_list = []
text_list = []
dic_list = []
article_tags = []
tag_id_counter = 0
for count, each_tag in enumerate(tags):
    tag = str(each_tag)[109:111]
    if str(tag)[1] == "}":
        tag = tag[0]
    # tag is our one or two-digit tag.
    tag_list.append(tag)
    tag_text = tags[count].text.strip()
    text_list.append(tag_text)

    # shove same-numbered tags into a list

    if tag == tag_id_counter:
        tag_text = tags[count].text.strip()
        article_tags.append(tag_text)
    else:
        tag_dict = {}
        tag_dict[f"{tag}"]: article_tags
        article_tags = []
        tag_id_counter += 1
tagframe["headline_num"] = tag_list
tagframe["tags"] = text_list

tags_dict = (
    tagframe.groupby("headline_num").apply(lambda x: [y for y in x["tags"]]).to_dict()
)
tags_dict = {**{str(k): None for k in range(len(headlines))}, **tags_dict}
del tags_dict["0"]

{'1': ['Jake Paul', 'Boxing', 'TMZ Sports'],
 '2': ['Taylor Swift',
  'Twitter',
  'Gossip / Rumors',
  'Controversial S#!T',
  'Music'],
 '3': ['Snoop Dogg',
  'Dave Chappelle',
  'Humor',
  'Hip Hop',
  'Drugs & Alcohol',
  'Comedy',
  'Music'],
 '4': ['Joe Biden', 'White House', 'Nurse!', 'Politix', 'Health'],
 '5': ['TV', 'Birthdays', 'Stranger Things', 'Viral', 'Music'],
 '6': ['Arnold Schwarzenegger',
  'RIP',
  'Family',
  'Movies',
  'TMZ Sports',
  'Birthdays'],
 '7': None,
 '8': ['Ted Cruz',
  'Jon Stewart',
  'Money',
  'Politix',
  'Exclusive',
  'Controversial S#!T',
  'Health',
  'Celebrity Feuds'],
 '9': ['Ivana Trump',
  'Donald Trump',
  'RIP',
  'Golf',
  'Celebrity Death',
  'TMZ Sports',
  'Hollywood Exes',
  'Politix',
  'Viral'],
 '10': ['Nelly Furtado', 'Drake', 'Hip Hop', 'Music'],
 '11': ['Britney Spears',
  'Money',
  'Family',
  'Exclusive',
  'Controversial S#!T',
  'Music'],
 '12': ['Nurse!', 'Instagram', 'Exclusive', 'Health'],
 '13': ['Nurse!', 'Health', 

In [183]:
{**{str(k): None for k in range(len(headlines) + 1)}, **tags_dict}

{'0': None,
 '1': ['Jake Paul', 'Boxing', 'TMZ Sports'],
 '2': ['Taylor Swift',
  'Twitter',
  'Gossip / Rumors',
  'Controversial S#!T',
  'Music'],
 '3': ['Snoop Dogg',
  'Dave Chappelle',
  'Humor',
  'Hip Hop',
  'Drugs & Alcohol',
  'Comedy',
  'Music'],
 '4': ['Joe Biden', 'White House', 'Nurse!', 'Politix', 'Health'],
 '5': ['TV', 'Birthdays', 'Stranger Things', 'Viral', 'Music'],
 '6': ['Arnold Schwarzenegger',
  'RIP',
  'Family',
  'Movies',
  'TMZ Sports',
  'Birthdays'],
 '7': None,
 '8': ['Ted Cruz',
  'Jon Stewart',
  'Money',
  'Politix',
  'Exclusive',
  'Controversial S#!T',
  'Health',
  'Celebrity Feuds'],
 '9': ['Ivana Trump',
  'Donald Trump',
  'RIP',
  'Golf',
  'Celebrity Death',
  'TMZ Sports',
  'Hollywood Exes',
  'Politix',
  'Viral'],
 '10': ['Nelly Furtado', 'Drake', 'Hip Hop', 'Music'],
 '11': ['Britney Spears',
  'Money',
  'Family',
  'Exclusive',
  'Controversial S#!T',
  'Music'],
 '12': ['Nurse!', 'Instagram', 'Exclusive', 'Health'],
 '13': ['Nurse!'

In [203]:
test = pd.DataFrame()
test["headline"] = headlines
test["tags"] = tags_dict.values()

In [204]:
test

,headline,tags
0,Jake Paul Hasim Rahman Jr. Fight Canceled ... ...,"[Jake Paul, Boxing, TMZ Sports]"
1,Taylor Swift Fan Claim She Donated to Environm...,"[Taylor Swift, Twitter, Gossip / Rumors, Contr..."
2,"Dave Chappelle Tokes Up, Performs with Snoop ....","[Snoop Dogg, Dave Chappelle, Humor, Hip Hop, D..."
3,President Biden I Got Covid Again ... After Da...,"[Joe Biden, White House, Nurse!, Politix, Health]"
4,Kate Bush's 'Wuthering Heights' Fans Gather En...,"[TV, Birthdays, Stranger Things, Viral, Music]"
5,Arnold Schwarzenegger Shout-Out to My Late Mot...,"[Arnold Schwarzenegger, RIP, Family, Movies, T..."
6,Stars and Scars You Be the judge.,None
7,Jon Stewart Calls BS on Ted Cruz ... You're Ly...,"[Ted Cruz, Jon Stewart, Money, Politix, Exclus..."
8,Ivana Trump Pics of Gravesite Go Viral ... Bar...,"[Ivana Trump, Donald Trump, RIP, Golf, Celebri..."
9,Drake Brings Out Nelly Furtado at OVO Fest ......,"[Nelly Furtado, Drake, Hip Hop, Music]"
